In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
sns.set_theme(style="whitegrid")

In [18]:
train_pd = pd.read_csv('data/train.csv')
test_pd = pd.read_csv('data/test.csv')

In [19]:
def encode_sex_column(data):
    encoder = OneHotEncoder()
    sex_encoded = encoder.fit_transform(data[["Sex"]])
    encoded_df = pd.DataFrame(sex_encoded.toarray(), columns=encoder.get_feature_names_out(["Sex"]))
    data = pd.concat([data.drop("Sex", axis=1), encoded_df], axis=1)
    return data

In [20]:
train_pd = encode_sex_column(train_pd)
test_pd = encode_sex_column(test_pd)

In [21]:
train_pd.head()

,id,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,1.0,0.0,0.0
1,1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,1.0,0.0,0.0
2,2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,0.0,1.0,0.0
3,3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0.0,0.0,1.0
4,4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,0.0,1.0,0.0


In [23]:
train_pd = train_pd.set_index("id")

In [25]:
test_pd = test_pd.set_index("id")

In [24]:
train_pd.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,1.0,0.0,0.0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,1.0,0.0,0.0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,0.0,1.0,0.0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0.0,0.0,1.0
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,0.0,1.0,0.0


In [29]:
def calculate_percentiles(data: pd.DataFrame, column: str):
    p01 = data[column].quantile(q=0.01)
    p99 = data[column].quantile(q=0.99)
    return p01, p99

In [30]:
for col in train_pd.columns:
    if col == "Rings":
        continue
    p01, p99 = calculate_percentiles(train_pd, col)
    print(col)
    print(p01, p99)

Length
0.19 0.72
Diameter
0.135 0.57
Height
0.045 0.215
Whole weight
0.032 1.93843
Whole weight.1
0.0125 0.8968600000000007
Whole weight.2
0.0065 0.422
Shell weight
0.01 0.57
Sex_F
0.0 1.0
Sex_I
0.0 1.0
Sex_M
0.0 1.0


In [33]:
train_pd.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99])

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794,0.292391,0.365204,0.342405
std,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221,0.454863,0.481490,0.474517
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000,0.000000,0.000000,0.000000
1%,0.190000,0.135000,0.045000,0.032000,0.012500,0.006500,0.010000,4.000000,0.000000,0.000000,0.000000
25%,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000,0.000000,0.000000,0.000000
50%,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000,0.000000,0.000000,0.000000
75%,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000,1.000000,1.000000,1.000000
99%,0.720000,0.570000,0.215000,1.938430,0.896860,0.422000,0.570000,20.000000,1.000000,1.000000,1.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000,1.000000,1.000000,1.000000


In [46]:
train_pd, val_pd = train_test_split(train_pd, test_size=0.2, random_state=42)

In [39]:
def save_to_numpy_file(data: pd.DataFrame, subset: str):
    if "Rings" in data.columns:
        y = data["Rings"].to_numpy()
        np.save(f"data/y_{subset}.npy", y)
        data = data.drop("Rings", axis=1)
    X = data.to_numpy()
    np.save(f"data/X_{subset}.npy", X)

In [47]:
save_to_numpy_file(train_pd, "train")

In [48]:
save_to_numpy_file(val_pd, "val")

In [41]:
save_to_numpy_file(test_pd, "test")